# 第17章 迭代器、生成器和经典协程

## 17.1 序列可迭代的原因：iter函数

### 17.1.1 Sentence类第1版：单词序列

In [1]:
import re
import reprlib

RE_WORD = re.compile(r'\w+')


class SentenceV1:

    def __init__(self, text):
        self.text = text
        # 使用正则表达式分割单词
        self.words = RE_WORD.findall(text)

    def __getitem__(self, index):
        return self.words[index]  

    def __len__(self):  
        return len(self.words)

    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text) 

In [2]:
s = SentenceV1('"The time has com," the Walrus said,')
s

Sentence('"The time ha... Walrus said,')

In [3]:
for word in s:
    print(word, end=" ")

The time has com the Walrus said 

In [4]:
list(s)

['The', 'time', 'has', 'com', 'the', 'Walrus', 'said']

### 17.1.2 内置函数iter

内置函数`iter`的操作步骤：
1. 检查对象是否实现了`__iter__`方法，如果实现了就调用它，获取一个迭代器。
2. 如果没有实现，但是实现了`__getitem__`方法，那么`iter()`创建一个迭代器，尝试按索引（从0开始）获取项。
3. 如果尝试失败，则Python抛出`TypeError`异常，通常会提示`'C' object is not iterable`（C对象不可迭代），其中C是目标对象所属的类。

`iter()`两种形式：
1. 迭代器：传入两个参数为函数或任何可迭代对象创建迭代器，第一个参数必须是一个可迭代对象，重复调用产生值；第二个参数是哨符（标记值），如果可调用对象返回哨符，则抛出StopIteration，而不产生哨符。
2. 用于构建按块读取工具。

### 17.1.3 可迭代对象与迭代器

- 可迭代对象的定义： 使用内置的iter可以获取迭代器的对象。如果对象实现了能返回迭代器的`__iter__`方法，那么对象就是可迭代的。序列都可以迭代。实现了`__getitem__`方法，而且接受从0开始的索引，这种对象也是可以迭代的。

- 可迭代对象与迭代器之间的关系：Python从可迭代对象中获取迭代器。

## 17.2 为Sentence类实现`__iter__`方法

### 17.2.1 Sentence类第2版：经典迭代器

**改进方案：**
1. 删除`__getitem__`方法。
2. 添加`__iter__`方法，初始化`SentenceIterator`类，返回一个迭代器。
3. 添加`SentenceIterator`类，实现`__next__`和`__iter__`方法。

In [5]:
import re
import reprlib

RE_WORD = re.compile(r'\w+')


class SentenceV2:

    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)

    def __repr__(self):
        return f'Sentence({reprlib.repr(self.text)})'

    def __iter__(self):
        # 返回一个迭代器
        return SentenceIterator(self.words)


class SentenceIterator:

    def __init__(self, words):
        self.words = words  
        # 初始化索引
        self.index = 0

    def __next__(self):
        try:
            word = self.words[self.index]
        except IndexError:
            raise StopIteration()
        self.index += 1
        return word

    def __iter__(self):  
        return self

### 17.2.2 可迭代对象和迭代器的区别

- 可迭代对象有一个`__iter__`方法，每次都实例化一个新迭代器。
- 迭代器要实现`__next__`方法，返回单个元素，此外还要实现`__iter__`方法，返回迭代器本身。
- 迭代器也是可迭代对象，但是可迭代对象不是迭代器。

### 17.2.3 Sentence类第3版：生成器函数

**改进方案：**
1. 添加`__iter__`方法：遍历单词数组，使用yield创建生成器。

In [7]:
import re
import reprlib

RE_WORD = re.compile(r'\w+')


class SentenceV3:

    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)

    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)

    def __iter__(self):
        for word in self.words:
            # 产生当前的word
            yield word

### 17.2.4 生成器的工作原理

只要Python函数的主体中有`yield`关键字，该函数就是生成器函数。调用生成器函数，返回一个生成器对象。

生成器工作原理：
1. 生成器函数创建一个生成器对象，包装生成器函数的主体。
2. 把生成器对象传给`next()`函数时，生成器函数提前执行函数主体中的下一个`yield`语句，返回产出的值，并在函数主体的当前位置暂停。
3. 函数的主体返回时，Python创建的外层生成器对象抛出`StopIteration`异常。

## 17.3 惰性实现版本

### 17.3.1 Sentence类第4版：惰性生成器

**改进方案：**
1. 修改`__init__`方法，删除`words`的初始化。
2. 修改`__iter__`方法，用正则表达式分割句子，并用`yield`返回迭代对象。

In [9]:
import re
import reprlib

RE_WORD = re.compile(r'\w+')


class SentenceV4:

    def __init__(self, text):
        self.text = text 

    def __repr__(self):
        return f'Sentence({reprlib.repr(self.text)})'

    def __iter__(self):
        # 产出MatchObject实例
        for match in RE_WORD.finditer(self.text):  
            yield match.group()

### 17.3.2 Sentence类第5版：惰性生成器表达式

**改进方案：**
1. 修改`__iter__`方法：使用生成器表达式构建生成器对象。

In [10]:
import re
import reprlib

RE_WORD = re.compile(r'\w+')


class SentenceV5:

    def __init__(self, text):
        self.text = text

    def __repr__(self):
        return f'Sentence({reprlib.repr(self.text)})'

    def __iter__(self):
        return (match.group() for match in RE_WORD.finditer(self.text))

## 17.4 迭代器和生成器

- 迭代器：泛指实现了__next__方法的对象。迭代器用于生成供客户代码使用的数据，即客户代码通过`for`循环或其他迭代方式，或者直接在迭代器上调用``next(it)`驱动迭代器。
- 生成器：由Python编译器构建的迭代器。为了创建生成器，使用`yield`关键字得到生成器函数。生成器表达式是构建生成器对象的另一种方式。

## 17.5 等差数列生成器

### 17.5.1 自定义的等差数列类`ArithmeticProgression`

In [11]:
class ArithmeticProgression:

    def __init__(self, begin, step, end=None):
        self.begin = begin
        self.step = step
        self.end = end  # None -> "infinite" series

    def __iter__(self):
        result_type = type(self.begin + self.step)
        result = result_type(self.begin)
        forever = self.end is None
        while forever or result < self.end:
            yield result
            result += self.step

In [12]:
ap = ArithmeticProgression(0, 1, 3)
list(ap)

[0, 1, 2]

In [13]:
ap = ArithmeticProgression(1, .5, 3)
list(ap)

[1.0, 1.5, 2.0, 2.5]

### 17.5.2 使用`itertools`模块生成等差数列

In [14]:
import itertools

gen = itertools.takewhile(lambda n: n < 3, itertools.count(1, .5))
list(gen)

[1, 1.5, 2.0, 2.5]

In [15]:
def aritprog_gen(begin, step, end=None):
    first = type(begin + step)(begin)
    ap_gen = itertools.count(first, step)
    if end is None:
        return ap_gen
    return itertools.takewhile(lambda n: n < end, ap_gen)

### 17.5.3 标准库中的生成器函数

In [17]:
# 返回输入的可迭代对象中连续的重叠对
list(itertools.pairwise(range(7)))

[(0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6)]

In [18]:
# 分组函数
for char, group in itertools.groupby('LLLAAAGG'):
    print(char , '-->', list(group))

L --> ['L', 'L', 'L']
A --> ['A', 'A', 'A']
G --> ['G', 'G']


## 17.6 yield from：从子生成器中产出

yield from表达式是把一个生成器的工作委托给一个子生成器。

### 17.6.1 重新实现chain

In [19]:
def chain(*iterables):
    for it in iterables:
        for i in it:
            yield i

In [20]:
s = 'ABC'
r = range(3)
list(chain(s, r))

['A', 'B', 'C', 0, 1, 2]

In [21]:
# 用yield from重新实现chain
def chain(*iterables):
    for i in iterables:
        yield from i

In [22]:
list(chain(s, r))

['A', 'B', 'C', 0, 1, 2]

### 17.6.2 遍历树状结构

In [28]:
def tree(cls, dis_level, level=0):
    if level <= dis_level:
        yield cls.__name__, level
        for sub_cls in cls.__subclasses__():
            yield from tree(sub_cls, dis_level, level=level+1)


def display(cls, display_level=5):
    for cls_name, level in tree(cls, display_level - 1):
        indent = ' ' * 4 * level
        print(f'{indent}{cls_name}')    

In [29]:
# 显示2层异常层次结构
display(BaseException, 2)

BaseException
    Exception
    GeneratorExit
    SystemExit
    KeyboardInterrupt
    CancelledError
    BaseExceptionGroup


## 17.7 经典协程

经典协程的类型提示：`Generator[YieldType, SendType, ReturnType]`

### 17.7.1 示例：使用协程计算累计平均值

In [31]:
from collections.abc import Generator

def averager() -> Generator[float, float, None]:
    total = 0.0
    count = 0
    average = 0.0
    while True:
        # 暂停执行协程，返回结果
        term = yield average
        total += term
        count += 1
        average = total/count

In [32]:
coro_avg = averager()
next(coro_avg)

0.0

In [33]:
coro_avg.send(10)

10.0

In [34]:
coro_avg.send(30)

20.0

In [35]:
coro_avg.send(5)

15.0

### 17.7.2 返回项数和平均值

In [36]:
from collections.abc import Generator
from typing import Union, NamedTuple

class Result(NamedTuple):
    count: int  # type: ignore  
    average: float

class Sentinel: 
    def __repr__(self):
        return f'<Sentinel>'

STOP = Sentinel()  

SendType = Union[float, Sentinel] 

def averager2(verbose: bool = False) -> Generator[None, SendType, Result]:  
    total = 0.0
    count = 0
    average = 0.0
    while True:
        term = yield
        if verbose:
            print('received:', term)
        # 如果term为哨符，则跳出循环    
        if isinstance(term, Sentinel):
            break
        total += term  # <4>
        count += 1
        average = total / count
    # 返回结果    
    return Result(count, average)

In [37]:
def compute():
    # 协程终止的StopIteration异常时获取返回值
    res = yield from averager2(True)
    print('computed:', res)
    return res 

In [40]:
# 创建委托协程的对象
comp = compute()

In [41]:
for v in [None, 10, 20, 30, STOP]: 
    try:
        comp.send(v)  
    except StopIteration as exc:  
        result = exc.value

received: 10
received: 20
received: 30
received: <Sentinel>
computed: Result(count=3, average=20.0)


## 17.8 杂谈

Python中极简的迭代器接口：使用`__next__`方法完成迭代器的组合。

可插拔的生成器：
- 需求：把CDS/ISIS文件转换成适合导入CouchDB或MongoDB的JSON文件。
- 演变：
    1. 第1版：使用CDS/ISIS导出的ISO-2709格式读取文件，采用渐进方式。
    2. 第1版问题：需要让脚本支持另一种数据格式（.mst文件），脚本需要接受多个命令行选项，用于调整输出的记录结构。
    3. 第2版：隔离读取逻辑，将其写进一对生成器函数中，一个函数支持一种输入格式。
- 总结：用生成器函数解耦了读逻辑和写逻辑，使用生成器，可以交叉读写，处理任意大小的文件。
- 扩展：如果需要再支持一种输入格式，只需要再添加一个生成器函数来实现读逻辑即可。